# Split-Apply-Combine czyli `groupby` w `pandas`

In [2]:
import pandas as pd
import scipy.stats as stats

Czasami dane badawcze trzeba zagregować i obliczyć proste statystyki deskryptywne. Funkcja `groupby` dostępna w `pandas` pozwala w tym celu wykorzystać metodologię "Split-Apply-Combine". Głównym jej założeniem jest to, że dużo operacji na danych można "rozłożyć" na trzy etapy:
* SPLIT - dzielimy dane na części według określonego kryterium (np. według warunku eksperymentalne)
* APPLY - do każdego z elementów podziału stosujemy jakąś procedurę (zazwyczaj redukującą wymiar danych) (np. obliczenie średniej)
* COMBINE - łączymy wyniki operacji w jedną strukturę danych

W naszym przykładzie będziemy analizować dane badawcze z eksperymentu, w którym badani mieli kliknąć wszystkie obiekty określonego koloru koloru na ekranie. W kolumnie `response` znajduje się liczba obiektów, które kliknęli. W badaniu były dwa rodzaje prób. W jednym rodzaju (`short` w kolumnie `condition`) obiekt opisany był za pomocą przymiotnika (np. "czerwone skrzypce"). W drugim rodzaju (`long` w kolumnie `condition`) obiekt opisany był za pomocą konstrukcji ze zdaniem złożonym (np. "skrzypce, które są czerwone). Dane znajdują się w pliku `descriptions.csv`.

In [6]:
data = pd.read_csv('descriptions.csv')
data.head()

,Unnamed: 0,participant,condition,item,response
0,0,1,short,trial_n1,3
1,1,1,long,trial_n2,7
2,2,1,short,trial_n3,4
3,3,1,long,trial_n4,5
4,4,1,short,trial_n5,3


Chcielibyśmy zobaczyć jaka jest średnia liczba klikniętych przedmiotów w obu warunkach eksperymentalnych. Żeby to sprawdzić posłużymy się funkcją `groupby`. W funkcji tej wskazujemy, według jakiego kryterium `pandas` ma podzielić nasze dane. Jeśli przekażemy nazwę kolumny (u nas `condition`), to podzieli dane na dwie części (to jest krok SPLIT).

Funkcja ta zwraca obiekt klasy `groupby`. W zasadzie większość operacji, które moglibyśmy wykonać na ramce danych możemy wykonać także na takim "pogrupowanym" obiekcie. Dla przykładu tutaj wybraliśmy jedną z kolumn (`response`) i obliczyliśmy średnią (to jest krok APPLY).

Proszę zwrócić uwagę, że zamiast jednej wartości dostaliśmy ramkę danych z 2 wartościami (to jest krok COMBINE).

In [43]:
data.groupby('condition')[['response']].mean()

,response
condition,
long,3.887500
short,4.004167


Moglibyśmy pogrupować nasze dane w inny sposób, np. według próby. Możemy mieć podejrzenie, że w późniejszych próbach badani będą wskazywać więcej obiektów (efekt wytrenowania). Możemy szybko obejrzeć i przeanalizować "wizualnie" dane pod tym względem, wywołując na odpowiednio "pogrupowanej" ramce danych metodę `describe`.

In [44]:
data.groupby('item')[['response']].describe()

response                                               
             count      mean       std  min   25%  50%   75%  max
item                                                             
trial_n1      24.0  3.916667  1.442120  1.0  3.00  4.0  5.00  7.0
trial_n10     24.0  4.666667  1.685402  2.0  3.75  4.0  6.00  8.0
trial_n11     24.0  3.750000  1.359348  1.0  3.00  4.0  4.25  6.0
trial_n12     24.0  3.791667  1.215092  2.0  3.00  4.0  5.00  6.0
trial_n13     24.0  3.958333  2.053188  1.0  2.75  3.0  5.25  8.0
trial_n14     24.0  3.708333  1.756458  1.0  2.75  4.0  4.00  9.0
trial_n15     24.0  3.875000  1.776966  1.0  3.00  4.0  5.00  8.0
trial_n16     24.0  3.416667  1.348644  1.0  2.75  4.0  4.25  5.0
trial_n17     24.0  4.083333  1.442120  1.0  3.75  4.0  5.00  6.0
trial_n18     24.0  4.166667  1.239448  2.0  3.75  4.0  5.00  7.0
trial_n19     24.0  4.625000  1.663221  2.0  4.00  5.0  6.00  8.0
trial_n2      24.0  4.333333  1.761093  2.0  3.00  4.0  5.25  8.0
trial_n20     24.0  3.291667  1.301476  1.0  2.75  3.0  4.00  6.0
trial_n3      24.0  3.875000  1.596532  0.0  3.00  4.0  5.00  7.0
trial_n4      24.0  4.208333  1.318074  2.0  3.00  4.0  5.00  7.0
trial_n5      24.0  4.125000  1.392917  2.0  3.00  4.0  5.00  7.0
trial_n6      24.0  3.833333  1.403928  2.0  3.00  4.0  5.00  7.0
trial_n7      24.0  4.000000  1.351328  1.0  3.00  4.0  5.00  7.0
trial_n8      24.0  3.458333  1.444003  2.0  2.00  3.0  4.00  7.0
trial_n9      24.0  3.833333  1.785611  1.0  2.75  4.0  5.00  9.0

Za pomocą `groupby` możemy wykonywać dużo bardziej skomplikowane operacje. Załóżmy, że bardziej niż średnia liczba wskazanych obiektów przez uczestnika w obu warunkach interesują nas mediany. Takie mediany chcielibyśmy porównywać między warunkami. Aby to zrobić musimy najpierw
1. Pogrupować dane według przecięcia dwóch kryteriów - numeru uczestnika i warunku w eksperymencie
2. Dla każdych 10 obserwacji, które są rezultatem takiego podziału, obliczyć medianę.
3. Pogrupować te mediany według kryterium warunku eksperymentalnego.
4. Zobaczyć statystyki deskryptywne dla dwóch uzyskanych rozkładów.

In [45]:
(data.groupby(['condition', 'participant'])['response'] # grupujemy według warunku i uczestnika 
.median() # obliczamy medianę
.groupby('condition') # ponownie grupujemy według warunku
.describe()) # obliczamy statystyki deskryptywne

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
long,24.0,3.708333,0.569452,2.5,3.375,4.0,4.0,5.0
short,24.0,3.895833,0.589384,3.0,3.500,4.0,4.0,5.0


Możliwości i funkcje `groupby` są bardzo, bardzo duże. Gorąco zachecam do eksperymentowania!